In [1]:
# Using TF Hub requires particular version of TF
!pip uninstall tensorflow
!pip install tensorflow==1.15.0

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 412.3 MB 22 kB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.8 MB 21.6 MB/s 
     |████████████████████████████████| 503 kB 54.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2d69aace5ae3c1ebdfea70de4148c6d95ed1066047a02d9e7548addc255f98da
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32

In [2]:
import tensorflow as tf
from bs4 import BeautifulSoup
print(tf.__version__)

1.15.0


In [7]:
# Load the file to your google Drive
# data in sgm can be downloaded from below link:
# http://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

from google.colab import drive 
drive.mount('/content/drive',force_remount=True)
import os 


documents = []
for file in os.listdir("/content/drive/My Drive/reuters_data/"):
    
    if file.endswith(".sgm"):
        
        # for each sgm file, read it
        # Update your file path below if you are mounting for google drive
        # It will ask for gmail login id and password details
        filename = os.path.join("/content/drive/My Drive/reuters_data/", file)
        f = open(filename, 'r', encoding='utf-8', errors='ignore')
        dataFile = f.read()
        
        # pass it to BeautifulSoup
        soup = BeautifulSoup(dataFile, 'html.parser')
        contents = soup.findAll('body')
        
        # for each body tag, extract it's text
        for content in contents:
            documents.append(content.text)
print('We have {} documents'.format(len(documents)))
print(documents[0])

Mounted at /content/drive
We have 19043 documents
Showers continued throughout the week in
the Bahia cocoa zone, alleviating the drought since early
January and improving prospects for the coming temporao,
although normal humidity levels have not been restored,
Comissaria Smith said in its weekly review.
    The dry period means the temporao will be late this year.
    Arrivals for the week ended February 22 were 155,221 bags
of 60 kilos making a cumulative total for the season of 5.93
mln against 5.81 at the same stage last year. Again it seems
that cocoa delivered earlier on consignment was included in the
arrivals figures.
    Comissaria Smith said there is still some doubt as to how
much old crop cocoa is still available as harvesting has
practically come to an end. With total Bahia crop estimates
around 6.4 mln bags and sales standing at almost 6.2 mln there
are a few hundred thousand bags still in the hands of farmers,
middlemen, exporters and processors.
    There are doubts as 

In [8]:
import re

def removeLinks(text):
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    return text

def removeHTMLTags(text):
    text = re.sub(r'<.*?>', '', text, flags=re.MULTILINE)
    return text

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def convertToLowerCase(text):
    return text.lower()

def removeWordsWithNumbers(text):
    return re.sub(r"\S*\d\S*", "", text).strip()

def removePuctuations(text):
    return re.sub(r'[^A-Za-z0-9]+', ' ', text)

def removeWhiteSpaces(text):
    return text.strip()


documents = list(map(removeLinks, documents))
documents = list(map(convertToLowerCase, documents))
documents = list(map(removeWordsWithNumbers, documents))
documents = list(map(removeHTMLTags, documents))
documents = list(map(decontracted, documents))
documents = list(map(removePuctuations, documents))
documents = list(map(removeWhiteSpaces, documents))
documents[0]

'showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february were bags of kilos making a cumulative total for the season of mln against at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around mln bags and sales standing at almost mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers are now experiencing dificulties in obtaining 

In [9]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [10]:
# Uses GPU or CPU?
tf.test.gpu_device_name()

''

In [11]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [12]:
%%time
# Restricting to 10 docs to enable faster computation
elmo_docs = [elmo_vectors([x]) for x in documents[:10]]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 2min 8s, sys: 3.62 s, total: 2min 11s
Wall time: 1min 21s


In [13]:
# Optional: Save the Embeddings if you want to avoid re-computation
import pickle
pickle_out = open("/content/drive/My Drive/reuters_data/elmo_train_reuters.pickle","wb")
pickle.dump(elmo_docs, pickle_out)
pickle_out.close()

In [14]:
# Optional: Load the saved embeddings
import pickle
pickle_in = open("/content/drive/My Drive/reuters_data/elmo_train_reuters.pickle", "rb")
elmo_docs_loaded = pickle.load(pickle_in)

In [15]:
# Find cosine similarity score

import numpy as np
from numpy import linalg as LA

def find_cosine_score(x1, x2):
  return np.dot(x1, x2.T) / LA.norm(x2)

# Find all similar documents for the index = '0' 
doc_idx = 0
docA = elmo_docs[doc_idx]
docA = docA/LA.norm(docA)

sim_score = {}
for i in range(len(elmo_docs)):
  docB = elmo_docs[i]
  sim_score[i] = find_cosine_score(docA, docB) 

In [16]:
# Sort in Descending scores
x = sorted(sim_score.items(), key=lambda x:x[1], reverse=True)[:5]
x

[(0, array([[1.]], dtype=float32)),
 (5, array([[0.8490526]], dtype=float32)),
 (4, array([[0.7933567]], dtype=float32)),
 (7, array([[0.7454589]], dtype=float32)),
 (3, array([[0.74164486]], dtype=float32))]

In [17]:
# Print the Results
for i in range(len(x)):
  print(f"-------------Document {i}-----------------") 
  print(documents[x[i][0]])

-------------Document 0-----------------
showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february were bags of kilos making a cumulative total for the season of mln against at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around mln bags and sales standing at almost mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers are no